# Overview

I tried to use a CFG feature grammar with lambda calculus to parse a natural language queries into a first order logic 
representation. My plan was to use the FOL as intermediate representation from which I could translate into other query 
languages such as Xpath or SQL. While I was successful in building gramars that achieved this for simple queries 
the translation from FOL to Xpath or SQL proved to difficult in the given time frame. I therefore returned to my first approach
of parsing NL queries directly into Xpath.

# Attempting Context Free Feature Grammar with Lambda Calculus

In [3]:
import nltk
from nltk import grammar, parse

sents = ['which planets have a radius of number']

g = """
% start S
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

VP[NUM=?n,SEM=?obj] -> TV[NUM=?n] NP[SEM=?obj]

NP[+INT,NUM=?n,SEM=<?det(?nom)>] -> Det[+INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[-INT,NUM=?n,SEM=?nom] -> Det[-INT, NUM=?n]  Nom[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?pp(?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
PP[SEM=?u] -> P[-LOC] UNIT[SEM=?u]

N[NUM=sg,SEM=<\\x.planet(x)>] -> 'planet'
N[NUM=pl,SEM=<\\x.planet(x)>] -> 'planets'
N[NUM=sg,SEM=<\\x y.radius(x,y)>] -> 'radius'
UNIT[SEM=<\\P x.P(x,number)>] -> 'number'

Det[+INT,NUM=pl,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[+INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[-INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'a'

TV[NUM=sg,SEM="",TNS=pres] -> 'has'
TV[NUM=pl,SEM="",TNS=pres] -> 'have'

P[-LOC,SEM=""] -> 'of'
"""
gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=2)
#trees = list(parser.parse(sents[0].split()))

for results in nltk.interpret_sents(sents, gram):
    for (synrep, semrep) in results:
        print(semrep)

exists x.(planet(x) & radius(x,number))


In [4]:
import nltk
from nltk import grammar, parse

sents = ['which planets have a radius of greater #NUM#']

g = """
% start S
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

VP[NUM=?n,SEM=?obj] -> TV[NUM=?n] NP[SEM=?obj]

NP[+INT,NUM=?n,SEM=<?det(?nom)>] -> Det[+INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[-INT,NUM=?n,SEM=?nom] -> Det[-INT, NUM=?n]  Nom[NUM=?n,SEM=?nom]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?nom(?pp)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[SEM=<?u(?a)>] -> A[SEM=?a] UNIT[SEM=?u]

PP[SEM=?nom] -> P[-LOC] Nom[SEM=?nom]

N[NUM=sg,SEM=<\\x.planet(x)>] -> 'planet'
N[NUM=pl,SEM=<\\x.planet(x)>] -> 'planets'
N[NUM=sg,SEM=<\\P x .exists y.(radius(x,y) & P(y))>] -> 'radius'
UNIT[SEM=<\\P.P(number)>] -> '#NUM#'
A[SEM=<\\x y.greater(x,y)>] -> 'greater'

Det[+INT,NUM=pl,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[+INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[-INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'a'

TV[NUM=sg,SEM="",TNS=pres] -> 'has'
TV[NUM=pl,SEM="",TNS=pres] -> 'have'

P[-LOC,SEM=""] -> 'of'

"""
gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=0)
trees = list(parser.parse(sents[0].split()))

for results in nltk.interpret_sents(sents, gram):
    for (synrep, semrep) in results:
        print(semrep)

exists x.(planet(x) & exists y.(radius(x,y) & greater(number,y)))


In [5]:
import nltk
from nltk import grammar, parse

sents = ['which planets have a radius of greater #NUM#']

g = """
% start S
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]

NP[+INT,NUM=?n,SEM=<?det(?nom)>] -> Det[+INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[-INT,NUM=?n,SEM=<?det(?nom)>] -> Det[-INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?pp(?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[SEM=<?u(?a)>] -> A[SEM=?a] UNIT[SEM=?u]
Nom[SEM=?a] -> A[SEM=?a]

PP[SEM=<?p(?nom)>] -> P[-LOC,SEM=?p] Nom[SEM=?nom]

N[NUM=sg,SEM=<\\x.planet(x)>] -> 'planet'
N[NUM=pl,SEM=<\\x.planet(x)>] -> 'planets'
N[NUM=sg,SEM=<\\x.radius(x)>] -> 'radius'
UNIT[SEM=<\\P.P(number)>] -> '#NUM#'
A[SEM=<\\x.great(x)>] -> 'great'
A[SEM=<\\x y.greater(x,y)>] -> 'greater'

Det[+INT,NUM=pl,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[+INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[-INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'a'

TV[NUM=sg,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'has'
TV[NUM=pl,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'have'

P[-LOC,SEM=<\\P \\Q x.(P(x) & Q(x))>] -> 'of'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=2)
#trees = list(parser.parse(sents[0].split()))

for results in nltk.interpret_sents(sents, gram):
    for (synrep, semrep) in results:
        print(semrep)

exists x.(planet(x) & exists z6.(greater(number,z6) & radius(z6) & have(x,z6)))


In [6]:
import nltk
from nltk import grammar, parse

sents = ['which planets have a radius of greater #NUM#']

g = """
% start S
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]

NP[+INT,NUM=?n,SEM=<?det(?nom)>] -> Det[+INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[-INT,NUM=?n,SEM=<?det(?nom)>] -> Det[-INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?pp(?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[SEM=<?u(?a)>] -> A[SEM=?a] UNIT[SEM=?u]
Nom[SEM=?a] -> A[SEM=?a]

PP[SEM=<?p(?nom)>] -> P[-LOC,SEM=?p] Nom[SEM=?nom]

N[NUM=sg,SEM=<\\x.planet(x)>] -> 'planet'
N[NUM=pl,SEM=<\\x.planet(x)>] -> 'planets'
N[NUM=sg,SEM=<\\x.radius(x)>] -> 'radius'
UNIT[SEM=<\\P.P(number)>] -> '#NUM#'
A[SEM=<\\x.great(x)>] -> 'great'
A[SEM=<\\x y.greater(x,y)>] -> 'greater'

Det[+INT,NUM=pl,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[+INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[-INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'a'

TV[NUM=sg,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'has'
TV[NUM=pl,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'have'

P[-LOC,SEM=<\\P \\Q x.(P(x) & Q(x))>] -> 'of'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=2)
#trees = list(parser.parse(sents[0].split()))

for results in nltk.interpret_sents(sents, gram):
    for (synrep, semrep) in results:
        print(semrep)

exists x.(planet(x) & exists z10.(greater(number,z10) & radius(z10) & have(x,z10)))


In [7]:
import nltk
from nltk import grammar, parse

sents = ['which planets have a radius of greater #NUM#']

g = """
% start S
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]

NP[+INT,NUM=?n,SEM=<?det(?nom)>] -> Det[+INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[-INT,NUM=?n,SEM=<?det(?nom)>] -> Det[-INT, NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?pp(?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[SEM=<?u(?a)>] -> A[SEM=?a] UNIT[SEM=?u]
Nom[SEM=?a] -> A[SEM=?a]

PP[SEM=<?p(?nom)>] -> P[-LOC,SEM=?p] Nom[SEM=?nom]

N[NUM=sg,SEM=<\\x.planet(x)>] -> 'planet'
N[NUM=pl,SEM=<\\x.planet(x)>] -> 'planets'
N[NUM=sg,SEM=<\\x.radius(x)>] -> 'radius'
UNIT[SEM=<\\P.P(number)>] -> '#NUM#'
A[SEM=<\\x.great(x)>] -> 'great'
A[SEM=<\\x y.greater(x,y)>] -> 'greater'

Det[+INT,NUM=pl,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[+INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'which'
Det[-INT,NUM=sg,SEM=<\\P \\Q.exists x.(P(x) & Q(x))>] -> 'a'

TV[NUM=sg,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'has'
TV[NUM=pl,TNS=pres,SEM=<\\X y.(X(\\x.have(y, x)))>] -> 'have'

P[-LOC,SEM=<\\P \\Q x.(P(x) & Q(x))>] -> 'of'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=2)
#trees = list(parser.parse(sents[0].split()))

for results in nltk.interpret_sents(sents, gram):
    for (synrep, semrep) in results:
        print(semrep) 

exists x.(planet(x) & exists z14.(greater(number,z14) & radius(z14) & have(x,z14)))


# Returning to the Simpler Approach

I decided to build the Xpath queries directly from the parse trees. However, I refined the CFG to be based on X-Bar syntax and strictly in Chomsky Normal Form. Here a first attempt with example syntactic tree:

In [ ]:
from nltk import grammar, parse
import nltk,urllib.request, gzip, io
from nltk import grammar, parse
from nltk.tokenize import word_tokenize
import re
from lxml import etree

In [8]:

sents = ['what planets have a mass of 19.4','which planets have a mass of 19.4',
'what planets have a radius of 0.188','which planets have a mass of at least 19.4',
'which planets have a mass of at most 0.001','which planets have a mass smaller than 0.001',
'which planets have a mass greater than 19.4','what planets were discovered before 2001',
'what planets were discovered after 2021','which planets have a mass smaller than 0.001',
'which planets have a mass smaller than 0.001 and a radius of less than 0.188',
'what planets have a mass larger than 0.001 and a radius of at least 0.188 and a mass of at least 0.188',
'what planets have a mass larger than 0.001 and have a radius of at least 0.188']

g = """
% start S
S[SEM=(?n + '[' + ?v+']')] -> N[BAR=2,SEM=?n] V[BAR=2,SEM=?v]

V[BAR=2,SEM=(?v + ?n)] -> V[BAR=1,SEM=?v] N[BAR=2,SEM=?n]
V[BAR=2,SEM=(?v + ?p)] -> V[BAR=1,SEM=?v] P[BAR=2,SEM=?p]
V[BAR=2,SEM=(?v + ?c)] -> V[BAR=1,SEM=?v] CONJ[BAR=2,SEM=?c]
V[BAR=1,SEM=?p] -> AUX PART[SEM=?p]
V[BAR=1,SEM=?v] -> V[BAR=0,SEM=?v]

CONJ[BAR=2,SEM=(?n + ?c)] -> N[BAR=2,SEM=?n] CONJ[BAR=1,SEM=?c]
CONJ[BAR=2,SEM=(?v + ?c)] -> V[BAR=2,SEM=?v] CONJ[BAR=1,SEM=?c]
CONJ[BAR=1,SEM=(?c+ ?cp)] -> CONJ[BAR=0,SEM=?c] CONJ[BAR=2,SEM=?cp] 
CONJ[BAR=1,SEM=(?c + ?n)] -> CONJ[BAR=0,SEM=?c] N[BAR=2,SEM=?n]
CONJ[BAR=1,SEM=(?c + ?v)] -> CONJ[BAR=0,SEM=?c] V[BAR=2,SEM=?v]

N[BAR=2,SEM=(?det + ?n)] -> Art[SEM=?det] N[BAR=1,SEM=?n] | Int[SEM=?det] N[BAR=1,SEM=?n]
N[BAR=2,SEM=(?a + ?n)] -> A[BAR=2,SEM=?a] N[BAR=1,SEM=?n]
N[BAR=1,SEM=(?n + ?p)] -> N[BAR=0,SEM=?n] P[-TIME, BAR=2,SEM=?p]
N[BAR=1,SEM=(?n + ?a)] -> N[BAR=0,SEM=?n] A[BAR=2,SEM=?a]
N[BAR=2,SEM=?n] -> N[BAR=1,SEM=?n]
N[BAR=1,SEM=?n] -> N[BAR=0,SEM=?n]

P[BAR=2,SEM=(?p + ?n)] -> P[BAR=1,SEM=?p] A[BAR=2,SEM=?n]
P[BAR=2,SEM=(?p + ?n)] -> P[BAR=1,SEM=?p] N[BAR=2,SEM=?n]
P[BAR=1,SEM=?p] -> P[BAR=0,SEM=?p]

A[BAR=2,SEM=(?a+?n)] -> A[BAR=1,SEM=?a] N[BAR=1,SEM=?n]
A[BAR=1,SEM=?a] -> A[BAR=0,SEM=?a]
A[BAR=1,SEM=?a] -> A[BAR=0,SEM=?a] CONJ[BAR=0]
A[BAR=1,SEM=?a] -> P[BAR=1] A[BAR=1,SEM=?a]

PART[SEM='discoveryyear'] -> 'discovered'
Int[SEM='.//'] -> 'which' | 'what'
AUX -> 'were'
V[BAR=0,SEM=''] -> 'have' | 'possess'
Art[SEM=''] -> 'a'
N[BAR=0,SEM='mass'] -> 'mass'
N[BAR=0,SEM='radius'] -> 'radius'
N[BAR=0,SEM='planet'] -> 'planets'
N[BAR=0,SEM="=#NUM#"] -> '#NUM#'
P[-TIME, BAR=0,SEM=''] -> 'of' |'at'
P[-TIME, BAR=0,SEM='<'] -> 'before'
P[+TIME, BAR=0,SEM='>'] -> 'after'
CONJ[BAR=0,SEM=''] -> 'than'
CONJ[BAR=0,SEM=' and '] -> 'and'
A[BAR=0,SEM='>'] -> 'bigger' | 'larger' | 'greater' | 'more' | 'least'
A[BAR=0,SEM='<'] -> 'smaller' | 'less' | 'most'
"""

sents = [re.sub(r'\d+\.*\d*','#NUM#',s).split() for s in sents]

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

trees = [list(parser.parse(s)) for _,s in enumerate(sents)]
#trees = list(parser.parse(sents[0].split()))

from nltk.draw.tree import draw_trees
draw_trees(trees[-1][0])

The (almost) finished version has more rules and tow different sentence types. It becomes apparent that the number of trees is exploding fast.
I am printing the results of the parser at the end of this code chunk:

In [10]:
g = """
% start S

S[SEM=?s] -> S1[SEM=?s] | S2[SEM=?s]
S1[SEM=('(.//' + ?n + ')' + '[position()<' + ?count + ']')] -> N[BAR=2,NUM=?num,SEM=?n,COUNT=?count]
S1[NUM=?num,SEM=(?s+ ';' + ?c)] -> S1[NUM=?num,SEM=?s] CONJ[BAR=1,SEM=?c]

S2[SEM=(?n + '[' + ?v+']'),NUM=?num] -> N[BAR=2,SEM=?n,NUM=?num] V[BAR=2,SEM=?v,NUM=?num]

V[BAR=2,SEM=(?v + ?n),NUM=?num] -> V[BAR=1,SEM=?v,NUM=?num] N[BAR=2,SEM=?n]
V[BAR=2,SEM=(?v + ?p),NUM=?num] -> V[BAR=1,SEM=?v,NUM=?num] P[BAR=2,SEM=?p,SUBC=?subc]
V[BAR=2,SEM=(?v + ?c),NUM=?num] -> V[BAR=1,SEM=?v,NUM=?num] CONJ[BAR=2,SEM=?c]
V[BAR=1,SEM=?p] -> AUX PART[SEM=?p]
V[BAR=1,SEM=?v,NUM=?num] -> V[BAR=0,SEM=?v,NUM=?num]

C[BAR=2,SEM=?v,NUM=?num] -> C[BAR=1] V[BAR=2,SEM=?v,NUM=?num]
C[BAR=2,SEM=?c] -> C[BAR=1] CONJ[BAR=2,SEM=?c]
C[BAR=1] -> C[BAR=0]

CONJ[BAR=2,SEM=(?n + ?c)] -> N[BAR=2,SEM=?n] CONJ[BAR=1,SEM=?c]
CONJ[BAR=2,SEM=(?v + ?c)] -> V[BAR=2,SEM=?v] CONJ[BAR=1,SEM=?c]
CONJ[BAR=2,SEM=(?p + ?c)] -> P[BAR=2,SEM=?p,SUBC=?subc] CONJ[BAR=1,SEM=?c]
CONJ[BAR=2,SEM=(?c + ?con)] -> C[BAR=2,SEM=?c] CONJ[BAR=1,SEM=?con]
CONJ[BAR=1,SEM=(?c+ ?cp)] -> CONJ[BAR=0,SEM=?c] CONJ[BAR=2,SEM=?cp] 
CONJ[BAR=1,SEM=(?c + ?n)] -> CONJ[BAR=0,SEM=?c] N[BAR=2,SEM=?n]
CONJ[BAR=1,SEM=(?c + ?n)] -> CONJ[BAR=0,SEM=?c] N[BAR=2,SEM=?n]
CONJ[BAR=1,SEM=(?c + ?v)] -> CONJ[BAR=0,SEM=?c] V[BAR=2,SEM=?v]
CONJ[BAR=1,SEM=(?c + ?v)] -> CONJ[BAR=0,SEM=?c] P[BAR=2,SEM=?v,SUBC=?subc]
CONJ[BAR=1,SEM=(?con + ?c)] -> CONJ[BAR=0,SEM=?con] C[BAR=2,SEM=?c]
CONJ[BAR=1,SEM=?s] -> CONJ[BAR=0,SEM=?c] S[SEM=?s]

N[BAR=2,NUM=?num,SEM=(?det + ?n),COUNT='=count(//*)'] -> Art[NUM=?num,SEM=?det] N[BAR=1,NUM=?num,SEM=?n] | Int[NUM=?num,SEM=?det] N[BAR=1,NUM=?num,SEM=?n]
N[BAR=2,NUM=?num,SEM=?n,COUNT=?count] -> Num[SEM=?count] N[BAR=1,NUM=?num,SEM=?n]
N[BAR=2,SEM=(?n + ' and ' + ?c),NUM=?num,COUNT='=count(//*)'] -> N[BAR=2,SEM=?n,NUM=?num] C[BAR=2,SEM=?c,NUM=?num]
N[BAR=2,SEM=(?a + ?n)] -> A[BAR=2,SEM=?a] N[BAR=1,SEM=?n]
N[BAR=1,SEM=(?n + ?p)] -> N[BAR=0,SEM=?n] P[BAR=2,SEM=?p,SUBC='-Adj']
N[BAR=1,SEM=(?n + '[' + ?p +']')] -> N[BAR=0,SEM=?n] P[BAR=2,SEM=?p,SUBC='+Adj']
N[BAR=1,SEM=(?n + '[' + ?c +']')] -> N[BAR=0,SEM=?n] C[BAR=2,SEM=?c]
N[BAR=1,SEM=(?n + ?a)] -> N[BAR=0,SEM=?n] A[BAR=2,SEM=?a]
N[BAR=1,NUM=?num,SEM=?n] -> N[BAR=0,NUM=?num,SEM=?n]

P[BAR=2,SEM=(?p + ?n),SUBC=?subc] -> P[BAR=1,SEM=?p,SUBC=?subc] A[BAR=2,SEM=?n]
P[BAR=2,SEM=(?p + ?n),SUBC=?subc] -> P[BAR=1,SEM=?p,SUBC=?subc] N[BAR=2,SEM=?n]
P[BAR=2,SEM=(?p + ?n),SUBC=?subc] -> P[BAR=1,SEM=?p,SUBC=?subc] Num[SEM=?n]
P[BAR=2,SEM=(?p + ?c),SUBC=?subc] -> P[BAR=1,SEM=?p,SUBC=?subc] CONJ[BAR=2,SEM=?c]
P[BAR=1,SEM=?p,SUBC=?subc] -> P[BAR=0,SEM=?p,SUBC=?subc]

A[BAR=2,SEM=(?a+?n)] -> A[BAR=1,SEM=?a] Num[SEM=?n]
A[BAR=1,SEM=?a] -> A[BAR=0,SEM=?a]
A[BAR=1,SEM=?a] -> A[BAR=0,SEM=?a] CONJ[BAR=0]
A[BAR=1,SEM=?a] -> P[BAR=1] A[BAR=1,SEM=?a]

PART[SEM='discoveryyear'] -> 'discovered'
Int[NUM='sg',SEM='.//'] -> 'which' | 'what'
Int[NUM='pl',SEM='.//'] -> 'which' | 'what'
AUX[NUM='pl'] -> 'were'
AUX[NUM='sg'] -> 'was'
V[NUM='pl',BAR=0,SEM=''] -> 'have' | 'possess'
V[NUM='sg',BAR=0,SEM=''] -> 'has' | 'possesses'
Art[NUM='sg',SEM=''] -> 'a' | 'any'
Art[NUM='pl',SEM=''] -> | 'any'
N[BAR=0,NUM='sg',SEM='mass'] -> 'mass'
N[BAR=0,NUM='sg',SEM='radius'] -> 'radius'
N[NUM='pl',BAR=0,SEM='planet'] -> 'planets'
N[NUM='sg',BAR=0,SEM='planet'] -> 'planet'
Num[NUM='na',SEM="=#NUM#"] -> '#NUM#'
Num[NUM='na',SEM="=#NUM0"] -> '#NUM0'
Num[NUM='na',SEM="=#NUM1"] -> '#NUM1'
Num[NUM='na',SEM="=#NUM2"] -> '#NUM2'
Num[NUM='na',SEM="=#NUM3"] -> '#NUM3'
P[BAR=0,SEM='',SUBC='-Adj'] -> 'of' |'at'
P[BAR=0,SEM='<',SUBC='-Adj'] -> 'before'
P[BAR=0,SEM='>',SUBC='-Adj'] -> 'after'
P[BAR=0,SEM='',SUBC='+Adj'] -> 'with'
C[BAR=0] -> 'that' | 'which'
CONJ[BAR=0,SEM=''] -> 'than'
CONJ[BAR=0,SEM=' and '] -> 'and'
A[BAR=0,SEM='>'] -> 'bigger' | 'larger' | 'greater' | 'more' | 'least'
A[BAR=0,SEM='<'] -> 'smaller' | 'less' | 'most' | 'maximally'
"""

queries = ['planets with a mass of 19.4','planets with a radius of 1',
           'planets with a radius of 1 and with a mass of 19.4',
           'planets with a mass of 19.4 and planets with a radius of 1',
          'planets with a mass of 19.4 and with a radius of 1 and planets with a radius of 1',
          'planets that were discovered before 2002','planets with a radius of 1 that were discovered before 2015',
          'what planets have a mass of 19.4','which planets have a mass of 19.4',
'what planets have a radius of 0.188','which planets have a mass of at least 19.4',
'which planets have a mass of at most 0.001','which planets have a mass smaller than 0.001',
'which planets have a mass greater than 19.4','what planets were discovered before 2001',
'what planets were discovered after 2021','which planets have a mass smaller than 0.001',
'which planets have a mass smaller than 0.001 and a radius of less than 0.188',
'what planets have a mass larger than 0.001 and a radius of at least 0.188 and a mass of at least 0.188',
'what planets have a mass larger than 0.001 and have a radius of at least 0.188','5 planets with a mass larger than 1 and 1 planet with a mass larger than 2',
'planets with a radius of 1 and with a mass of 19.4','planets that were discovered before 2001 and have a mass greater than 3']


gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)
for query in queries:
    query = re.sub(r'\d+\.*\d*','#NUM#',query)
    print('\n',query)
    trees = list(parser.parse(query.split()))
    for i,t in enumerate(trees):
        answer = trees[i].label()['SEM']
        print(i,answer)


 planets with a mass of #NUM#
0 ((.//, , planet, [, , , mass, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a radius of #NUM#
0 ((.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a radius of #NUM# and with a mass of #NUM#
0 ((.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ], ;,  and , , , mass, , =#NUM#)
1 ((.//, , planet, [, , , radius, , =#NUM#,  and , , , mass, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a mass of #NUM# and planets with a radius of #NUM#
0 ((.//, , planet, [, , , mass, , =#NUM#, ], ), [position()<, =count(//*), ], ;, (.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])
1 ((.//, , planet, [, , , mass, , =#NUM#, (.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ], ], ), [position()<, =count(//*), ])
2 ((.//, , planet, [, , , mass, , =#NUM#, ], ), [position()<, =count(//*), ], ;,  and , , planet, [, , , radius, , =#NUM#, ]

0 (.//, planet, [, , , mass, >, =#NUM#,  and , , , radius, >, =#NUM#, ])
1 (.//, planet, [, , , mass, >, =#NUM#,  and , , , radius, , >, =#NUM#, ])

 #NUM# planets with a mass larger than #NUM# and #NUM# planet with a mass larger than #NUM#
0 ((.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ], ;, (.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ])
1 ((.//, planet, [, , , mass, >, =#NUM#, (.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ], ], ), [position()<, =#NUM#, ])
2 ((.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ], ;,  and , planet, [, , , mass, >, =#NUM#, ])
3 ((.//, planet, [, , , mass, >, =#NUM#,  and , planet, [, , , mass, >, =#NUM#, ], ], ), [position()<, =#NUM#, ])

 planets with a radius of #NUM# and with a mass of #NUM#
0 ((.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ], ;,  and , , , mass, , =#NUM#)
1 ((.//, , planet, [, , , radius, , =#NUM#,  and , , , mass, , =#NUM#, ], )

# Putting it together with German

I now added in rules for German. For the most part, the rules for English could be reused because the syntactic patterns are similar for the queries that I use. The terminal nodes have to be duplicated and a 'de' tag has to be added to those. The language feature is added to all nodes such that it can later be used to select language-specific responses by the dialogue agent. A complicating factor is that German uses verb-final word order in relative clauses (e.g., 'a planet that a radius of 1 has') and I had to create a German-specific feature +VFINAL and introduce additional rules making use of this feature. The final Grammar looked like this:   

In [11]:
import re

g = """
% start S

S[SEM=?s,NUM=?num,L=?l] -> S1[SEM=?s,NUM=?num,L=?l] | S2[SEM=?s,NUM=?num,L=?l]
S1[SEM=('(.//' + ?n + ')' + '[position()<' + ?count + ']'),L=?l] -> N[BAR=2,NUM=?num,SEM=?n,COUNT=?count,L=?l]
S1[NUM=?num,SEM=(?s+ ';' + ?c),L=?l] -> S1[NUM=?num,SEM=?s,L=?l] CONJ[BAR=1,SEM=?c,L=?l]

S2[SEM=(?n + '[' + ?v+']'),NUM=?num,L=?l] -> N[BAR=2,SEM=?n,NUM=?num,L=?l] V[BAR=2,SEM=?v,NUM=?num,L=?l]
S2[NUM=?num,SEM=(?s+ ';' + ?c),L=?l] -> S2[NUM=?num,SEM=?s,L=?l] CONJ[BAR=1,SEM=?c,L=?l]

V[BAR=2,SEM=(?v + ?n),NUM=?num,L=?l] -> V[BAR=1,SEM=?v,NUM=?num,L=?l] N[BAR=2,SEM=?n,L=?l]
V[BAR=2,SEM=(?v + ?n),NUM=?num,L='de',+VFINAL] -> N[BAR=2,SEM=?n] V[BAR=1,SEM=?v,NUM=?num,L='de']
V[BAR=2,SEM=(?v + ?p),NUM=?num,L=?l] -> V[BAR=1,SEM=?v,NUM=?num,L=?l] P[BAR=2,SEM=?p,SUBC=?subc,L=?l]
V[BAR=2,SEM=(?v + ?p),NUM=?num,L='de',+VFINAL] -> P[BAR=2,SEM=?p,SUBC=?subc,L='de'] V[BAR=1,SEM=?v,NUM=?num,L='de',+VFINAL]
V[BAR=2,SEM=(?v + ?c),NUM=?num,L=?l] -> V[BAR=1,SEM=?v,NUM=?num,L=?l] CONJ[BAR=2,SEM=?c,L=?l]
V[BAR=1,SEM=?p] -> AUX PART[SEM=?p,L=?l]
V[BAR=1,SEM=?p,L='de',+VFINAL] -> PART[SEM=?p,L='de'] AUX[L='de']
V[BAR=1,SEM='',L='de'] -> AUX[L='de']
V[BAR=1,SEM=?v,NUM=?num,L=?l] -> V[BAR=0,SEM=?v,NUM=?num,L=?l]

C[BAR=2,SEM=?v,NUM=?num,L=?l] -> C[BAR=1,L=?l] V[BAR=2,SEM=?v,NUM=?num,L=?l]
C[BAR=2,SEM=?v,NUM=?num,L='de'] -> C[BAR=1,L='de'] V[BAR=2,SEM=?v,NUM=?num,L='de',+VFINAL]
C[BAR=2,SEM=?c,L=?l] -> C[BAR=1,L=?l] CONJ[BAR=2,SEM=?c,L=?l]
C[BAR=1,L=?l] -> C[BAR=0,L=?l]

CONJ[BAR=2,SEM=(?n + ?c),L=?l] -> N[BAR=2,SEM=?n,L=?l] CONJ[BAR=1,SEM=?c,L=?l]
CONJ[BAR=2,SEM=(?v + ?c),L=?l] -> V[BAR=2,SEM=?v,L=?l] CONJ[BAR=1,SEM=?c,L=?l]
CONJ[BAR=2,SEM=(?p + ?c),L=?l] -> P[BAR=2,SEM=?p,SUBC=?subc,L=?l] CONJ[BAR=1,SEM=?c,L=?l]
CONJ[BAR=2,SEM=(?c + ?con),L=?l] -> C[BAR=2,SEM=?c,L=?l] CONJ[BAR=1,SEM=?con,L=?l]
CONJ[BAR=1,SEM=(?c+ ?cp),L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] CONJ[BAR=2,SEM=?cp,L=?l] 
CONJ[BAR=1,SEM=(?c + ?n),L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] N[BAR=2,SEM=?n,L=?l]
CONJ[BAR=1,SEM=(?c + ?n),L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] N[BAR=2,SEM=?n,L=?l]
CONJ[BAR=1,SEM=(?c + ?v),L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] V[BAR=2,SEM=?v,L=?l]
CONJ[BAR=1,SEM=(?c + ?v),L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] P[BAR=2,SEM=?v,SUBC=?subc,L=?l]
CONJ[BAR=1,SEM=(?con + ?c),L=?l] -> CONJ[BAR=0,SEM=?con,L=?l] C[BAR=2,SEM=?c,L=?l]
CONJ[BAR=1,SEM=?s,L=?l] -> CONJ[BAR=0,SEM=?c,L=?l] S[SEM=?s,L=?l]

N[BAR=2,NUM=?num,SEM=(?det + ?n),COUNT='=count(//*)',L=?l] -> Art[NUM=?num,SEM=?det,L=?l] N[BAR=1,NUM=?num,SEM=?n,L=?l] | Int[NUM=?num,SEM=?det,L=?l] N[BAR=1,NUM=?num,SEM=?n,L=?l]
N[BAR=2,NUM=?num,SEM=?n,COUNT=?count,L=?l] -> Num[SEM=?count,L=?l] N[BAR=1,NUM=?num,SEM=?n,L=?l]
N[BAR=2,SEM=(?n + ' and ' + ?c),NUM=?num,COUNT='=count(//*)',L=?l] -> N[BAR=2,SEM=?n,NUM=?num,L=?l] C[BAR=2,SEM=?c,NUM=?num,L=?l]
N[BAR=2,SEM=(?a + ?n),L=?l] -> A[BAR=2,SEM=?a,L=?l] N[BAR=1,SEM=?n,L=?l]
N[BAR=1,SEM=(?n + ?p),L=?l] -> N[BAR=0,SEM=?n,L=?l] P[BAR=2,SEM=?p,SUBC='-Adj',L=?l]
N[BAR=1,SEM=(?n + '[' + ?p +']'),L=?l] -> N[BAR=1,SEM=?n,L=?l] P[BAR=2,SEM=?p,SUBC='+Adj',L=?l]
N[BAR=1,SEM=(?n + '[' + ?c +']'),L=?l] -> N[BAR=0,SEM=?n,L=?l] C[BAR=2,SEM=?c,L=?l]
N[BAR=1,SEM=(?n + ?a),L=?l] -> N[BAR=0,SEM=?n,L=?l] A[BAR=2,SEM=?a,L=?l]
N[BAR=1,NUM=?num,SEM=?n,L=?l] -> N[BAR=0,NUM=?num,SEM=?n,L=?l]

P[BAR=2,SEM=(?p + ?n),SUBC=?subc,L=?l] -> P[BAR=1,SEM=?p,SUBC=?subc,L=?l] A[BAR=2,SEM=?n,L=?l]
P[BAR=2,SEM=(?p + ?n),SUBC=?subc,L=?l] -> P[BAR=1,SEM=?p,SUBC=?subc,L=?l] N[BAR=2,SEM=?n,L=?l]
P[BAR=2,SEM=(?p + ?n),SUBC=?subc,L=?l] -> P[BAR=1,SEM=?p,SUBC=?subc,L=?l] Num[SEM=?n,L=?l]
P[BAR=2,SEM=(?s + ?p + ?n),SUBC=?subc,L='de'] -> P[BAR=1,SEM=?p,SUBC=?subc,L='de'] Num[SEM=?n] PART[SEM=?s,L='de']
P[BAR=2,SEM=(?p + ?c),SUBC=?subc,L=?l] -> P[BAR=1,SEM=?p,SUBC=?subc,L=?l] CONJ[BAR=2,SEM=?c,L=?l]

P[BAR=1,SEM=?p,SUBC=?subc,L=?l] -> P[BAR=0,SEM=?p,SUBC=?subc,L=?l]

A[BAR=2,SEM=(?a+?n),L=?l] -> A[BAR=1,SEM=?a,L=?l] Num[SEM=?n,L=?l]
A[BAR=1,SEM=?a,L=?l] -> A[BAR=0,SEM=?a,L=?l]
A[BAR=1,SEM=?a,L=?l] -> A[BAR=0,SEM=?a,L=?l] CONJ[BAR=0,L=?l]
A[BAR=1,SEM=?a,L=?l] -> P[BAR=1,L=?l] A[BAR=1,SEM=?a,L=?l]

PART[SEM='discoveryyear',L='en'] -> 'discovered'
Int[NUM='sg',SEM='.//',L='en'] -> 'which' | 'what'
Int[NUM='pl',SEM='.//',L='en'] -> 'which' | 'what'
AUX[NUM='pl',L='en'] -> 'were'
AUX[NUM='sg',L='en'] -> 'was'
V[NUM='pl',BAR=0,SEM='',L='en'] -> 'have' | 'possess'
V[NUM='sg',BAR=0,SEM='',L='en'] -> 'has' | 'possesses'
Art[NUM='sg',SEM='',L='en'] -> 'a' | 'any' | 'an'
Art[NUM='pl',SEM='',L='en'] -> | 'any'
N[BAR=0,NUM='sg',SEM='mass',L='en'] -> 'mass'
N[BAR=0,NUM='sg',SEM='radius',L='en'] -> 'radius'
N[BAR=0,NUM='sg',SEM='age',L='en'] -> 'age'
N[BAR=0,NUM='sg',SEM='temperature',L='en'] -> 'temperature'
N[NUM='pl',BAR=0,SEM='planet',L='en'] -> 'planets'
N[NUM='sg',BAR=0,SEM='planet',L='en'] -> 'planet'
Num[NUM='na',SEM="=#NUM#"] -> '#num#'
Num[NUM='na',SEM="=#NUM0"] -> '#num0'
Num[NUM='na',SEM="=#NUM1"] -> '#num1'
Num[NUM='na',SEM="=#NUM2"] -> '#num2'
Num[NUM='na',SEM="=#NUM3"] -> '#num3'
Num[NUM='na',SEM="=#NUM4"] -> '#num4'
Num[NUM='na',SEM="=#NUM5"] -> '#num5'
Num[NUM='na',SEM="=#NUM6"] -> '#num6'
Num[NUM='na',SEM="=#NUM7"] -> '#num7'
Num[NUM='na',SEM="=#NUM8"] -> '#num8'
Num[NUM='na',SEM="=#NUM9"] -> '#num9'
Num[NUM='na',SEM="=#NUM10"] -> '#num10'
P[BAR=0,SEM='',SUBC='-Adj',L='en'] -> 'of' |'at'
P[BAR=0,SEM='<',SUBC='-Adj',L='en'] -> 'before'
P[BAR=0,SEM='>',SUBC='-Adj',L='en'] -> 'after'
P[BAR=0,SEM='',SUBC='-Adj',L='en'] -> 'in'
P[BAR=0,SEM='',SUBC='+Adj',L='en'] -> 'with'
C[BAR=0,L='en'] -> 'that' | 'which'
CONJ[BAR=0,SEM='',L='en'] -> 'than'
CONJ[BAR=0,SEM=' and ',L='en'] -> 'and'
A[BAR=0,SEM='>',L='en'] -> 'bigger' | 'larger' | 'greater' | 'more' | 'least' | 'above' | 'over'
A[BAR=0,SEM='<',L='en'] -> 'smaller' | 'less' | 'most' | 'maximally' | 'below' | 'under'

PART[SEM='discoveryyear',L='de'] -> 'entdeckt'
Int[NUM='sg',SEM='.//',L='de'] -> 'welcher'
Int[NUM='pl',SEM='.//',L='de'] -> 'welche'
AUX[NUM='pl',L='de'] -> 'wurden'
AUX[NUM='sg',L='de'] -> 'wurde'
V[NUM='pl',BAR=0,SEM='',L='de'] -> 'haben' | 'besitzen'
V[NUM='sg',BAR=0,SEM='',L='de'] -> 'hat' | 'besitzt'
Art[NUM='sg',SEM='',L='de'] -> 'ein' | 'einen' | 'eine' | 'einem' | 'einer'
Art[NUM='pl',SEM='',L='de'] -> 
N[BAR=0,NUM='sg',SEM='mass',L='de'] -> 'masse'
N[BAR=0,NUM='sg',SEM='radius',L='de'] -> 'radius'
N[BAR=0,NUM='sg',SEM='age',L='de'] -> 'alter'
N[BAR=0,NUM='sg',SEM='temperature',L='en'] -> 'temperatur'
N[NUM='pl',BAR=0,SEM='planet',L='de'] -> 'planeten'
N[NUM='sg',BAR=0,SEM='planet',L='de'] -> 'planet' | 'planeten'
P[BAR=0,SEM='',SUBC='-Adj',L='de'] -> 'von'
P[BAR=0,SEM='<',SUBC='-Adj',L='de'] -> 'vor'
P[BAR=0,SEM='>',SUBC='-Adj',L='de'] -> 'nach'
P[BAR=0,SEM='',SUBC='-Adj',L='de'] -> 'in'
P[BAR=0,SEM='',SUBC='+Adj',L='de'] -> 'mit'
C[BAR=0,L='de'] -> 'der' | 'die'
CONJ[BAR=0,SEM='',L='de'] -> 'als'
CONJ[BAR=0,SEM=' and ',L='de'] -> 'und'
A[BAR=0,SEM='>',L='de'] -> 'größer' | 'mehr' | 'mindestens'
A[BAR=0,SEM='<',L='de'] -> 'kleiner' | 'weniger' | 'maximal' | 'unter'
"""

queries = ['planets with a mass of 19.4','planets with a radius of 1',
           'planets with a radius of 1 and with a mass of 19.4',
           'planets with a mass of 19.4 and planets with a radius of 1',
          'planets with a mass of 19.4 and with a radius of 1 and planets with a radius of 1',
          'planets that were discovered before 2002','planets with a radius of 1 that were discovered before 2015',
          'what planets have a mass of 19.4','which planets have a mass of 19.4',
'what planets have a radius of 0.188','which planets have a mass of at least 19.4',
'which planets have a mass of at most 0.001','which planets have a mass smaller than 0.001',
'which planets have a mass greater than 19.4','what planets were discovered before 2001',
'what planets were discovered after 2021','which planets have a mass smaller than 0.001',
'which planets have a mass smaller than 0.001 and a radius of less than 0.188',
'what planets have a mass larger than 0.001 and a radius of at least 0.188 and a mass of at least 0.188',
'what planets have a mass larger than 0.001 and have a radius of at least 0.188','5 planets with a mass larger than 1 and 1 planet with a mass larger than 2',
'planets with a radius of 1 and with a mass of 19.4','planets that were discovered before 2001 and have a mass greater than 3',
          'planeten die vor 1 entdeckt wurden und einen radius von 1 haben und eine masse von 1',
          'Gibt es Planeten, die vor 2001 entdeckt wurden?',
           'Show me 1 planet with an age of maximally 0.1 and a mass of at least 1 that was discovered in 2020 and 3 planets that have a radius of at least 1']

def normalize(query):
    query = query.lower()
    query = re.sub(r'[^\w\s#]','',query)
    query = re.sub(
        r'zeig mir|show me|are there any|are there|gibt es|can you show me|look for|search|suche?|finde?',
        '',query)
    return query

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram,trace=0)
for query in queries:
    query = re.sub(r'\d+\.*\d*','#NUM#',query)
    query = normalize(query)
    print('\n',query)
    trees = list(parser.parse(query.split()))
    for i,t in enumerate(trees):
        answer = trees[i].label()['SEM']
        print(i,answer)


 planets with a mass of #num#
0 ((.//, , planet, [, , , mass, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a radius of #num#
0 ((.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a radius of #num# and with a mass of #num#
0 ((.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ], ;,  and , , , mass, , =#NUM#)
1 ((.//, , planet, [, , , radius, , =#NUM#,  and , , , mass, , =#NUM#, ], ), [position()<, =count(//*), ])

 planets with a mass of #num# and planets with a radius of #num#
0 ((.//, , planet, [, , , mass, , =#NUM#,  and , , planet, ], [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])
1 ((.//, , planet, [, , , mass, , =#NUM#, (.//, , planet, ), [position()<, =count(//*), ], ], [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])
2 ((.//, , planet, [, , , mass, , =#NUM#, ], ), [position()<, =count(//*), ], ;, (.//, , planet, [, , , radius, , =#NUM#, ], ), [position()<, =count(//*), ])
3

0 (.//, planet, [, , , mass, >, =#NUM#, ], ;,  and , , , radius, >, =#NUM#)
1 (.//, planet, [, , , mass, >, =#NUM#,  and , , , radius, >, =#NUM#, ])
2 (.//, planet, [, , , mass, >, =#NUM#, ], ;,  and , , , radius, , >, =#NUM#)
3 (.//, planet, [, , , mass, >, =#NUM#,  and , , , radius, , >, =#NUM#, ])

 #num# planets with a mass larger than #num# and #num# planet with a mass larger than #num#
0 ((.//, planet, [, , , mass, >, =#NUM#,  and , planet, ], [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ])
1 ((.//, planet, [, , , mass, >, =#NUM#, (.//, planet, ), [position()<, =#NUM#, ], ], [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ])
2 ((.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ], ;, (.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ])
3 ((.//, planet, [, , , mass, >, =#NUM#, (.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NUM#, ], ], ), [position()<, =#NUM#, ])
4 ((.//, planet, [, , , mass, >, =#NUM#, ], ), [position()<, =#NU

0 ((.//, , planet,  and , , discoveryyear, <, =#NUM#, ), [position()<, =count(//*), ])
1 ((.//, , planet,  and , , discoveryyear, <, =#NUM#, ), [position()<, =count(//*), ])
2 ((.//, , planet, [, , discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
3 ((.//, , planet, [, , discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
4 ((.//, , planet, [, , discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
5 ((.//, , planet, [, , discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
6 ((.//, , planet,  and , discoveryyear, <, =#NUM#, ), [position()<, =count(//*), ])
7 ((.//, , planet,  and , discoveryyear, <, =#NUM#, ), [position()<, =count(//*), ])
8 ((.//, , planet, [, discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
9 ((.//, , planet, [, discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
10 ((.//, , planet, [, discoveryyear, <, =#NUM#, ], ), [position()<, =count(//*), ])
11 ((.//, , planet, [, discoveryyear, <, =#NUM#, ], ), [pos

815 ((.//, planet, [, , , age, <, =#NUM#, (.//, , mass, , >, =#NUM#,  and , discoveryyear, , =#NUM#, ), [position()<, =count(//*), ], ;,  and , planet, [, , , radius, , >, =#NUM#, ], ], ), [position()<, =#NUM#, ])
816 ((.//, planet, [, , , age, , <, =#NUM#, ], ), [position()<, =#NUM#, ], ;,  and , , mass, , >, =#NUM#,  and , discoveryyear, , =#NUM#,  and , planet, [, , , radius, , >, =#NUM#, ])
817 ((.//, planet, [, , , age, , <, =#NUM#, ], ), [position()<, =#NUM#, ], ;,  and , , mass, , >, =#NUM#,  and , discoveryyear, , =#NUM#,  and , planet, [, , , radius, , >, =#NUM#, ])
818 ((.//, planet, [, , , age, , <, =#NUM#, ], ), [position()<, =#NUM#, ], ;,  and , , mass, , >, =#NUM#,  and , discoveryyear, , =#NUM#,  and , planet, [, , , radius, , >, =#NUM#, ])
819 ((.//, planet, [, , , age, , <, =#NUM#,  and , , mass, , >, =#NUM#,  and , discoveryyear, , =#NUM#,  and , planet, [, , , radius, , >, =#NUM#, ], ], ), [position()<, =#NUM#, ])
820 ((.//, planet, [, , , age, , <, =#NUM#,  and , , 